## 3. Octave and one-third octave band filters

In this notebook we'll go through designing and appyling **fractional bandpass filtering** (mainly of 1/1 and 1/3 octave)** to our (already processed) input signal. 

This filters are one of 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (15,9)
sns.set_context('notebook', font_scale=1.2)